In [1]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from IPython.display import display, clear_output
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Settings and parameters

In [2]:
### Settings and parameters
LENGTH_X  = 400    # resolution x direction  # default 400
LENGTH_Y  = 100    # resolution y direction
RHO_0     = 1      # average density
TAU       = 0.6    # collision timescale (relaxation term)
# tau = 1.9739
# tau = 0.9
N_STEPS   = 4000   # number of timesteps
U_MAX     = 0.1    # maximum velocity of Poiseuille inflow
INLET_IDX = 0
OUTLET_IDX = LENGTH_X - 1
PIPE_LENGTH = LENGTH_Y - 2  # L
INLET_SL = np.s_[:, 0]
OUTLET_SL = np.s_[:, LENGTH_X - 1]

In [3]:
### Cylinder parameters
# X.shape: (100, 400) Y shape: (100, 400)
X, Y = np.meshgrid(range(LENGTH_X), range(LENGTH_Y))
# INFO: shape the same as all space, but only partially filled with cylinder
# cylinder shape: (100, 400)
CYLINDER_RADIUS = 4
# True within cylinder boundaries
CYLINDER_MASK = (X - LENGTH_X / 4) ** 2 + (Y - LENGTH_Y / 2) ** 2 < (LENGTH_Y // CYLINDER_RADIUS) ** 2
# pylab.imshow(CYLINDER_MASK, cmap='gray')

## Vector params

In [4]:
### Vectors params
# General params
LEFT_COL_NAMES = ["NW", "W", "SW"]
CENT_COL_NAMES = ["N", "C", "S"]
RIGHT_COL_NAMES = ["NE", "E", "SE"]
N_VECTORS = 9
VECTOR_INDEXES = np.arange(N_VECTORS)

### Old style

In [5]:
# Old style lattices definitions
#                                      0    1     2    3    4     5     6    7     8
#                                      C    N     NE   E    SE    S    SW    W     NW
VECTORS_VELOCITIES_X_OLD = np.array([  0,   0,    1,   1,   1,    0,   -1,  -1,   -1,  ])
VECTORS_VELOCITIES_Y_OLD = np.array([  0,   1,    1,   0,  -1,   -1,   -1,   0,    1,  ])
VECTORS_WEIGHTS_OLD = np.array([      4/9, 1/9, 1/36, 1/9, 1/36, 1/9, 1/36, 1/9, 1/36,  ]) # sums to 1
# NOTE: this should be updated to the new style
LAT_LEFT_COL_SL_OLD = np.s_[:, [8, 7, 6]]
LAT_CENT_COL_SL_OLD = np.s_[:, [1, 0, 5]]
LAT_RIGHT_COL_SL_OLD= np.s_[:, [2, 3, 4]]
VECTORS_DIRECTIONS_OLD = np.array("C N NE E SE S SW W NW".split())
assert np.all(VECTORS_DIRECTIONS_OLD[LAT_LEFT_COL_SL_OLD[1]] == LEFT_COL_NAMES)
assert np.all(VECTORS_DIRECTIONS_OLD[LAT_CENT_COL_SL_OLD[1]] == CENT_COL_NAMES)
assert np.all(VECTORS_DIRECTIONS_OLD[LAT_RIGHT_COL_SL_OLD[1]] == RIGHT_COL_NAMES)
                                # C  S  SW W  NW  
CYL_BOUNCE_BACK_DIRECTIONS_OLD = [0, 5, 6, 7, 8, 1, 2, 3, 4]

### New style

In [6]:
# New style lattices definitions
VECTORS_VELOCITIES_X_NEW = np.array([
    [-1, 0, 1,],
    [-1, 0, 1,],
    [-1, 0, 1,],
]).reshape(-1)
VECTORS_VELOCITIES_Y_NEW = np.array([
     [1,  1,  1,],
     [0,  0,  0,],
    [-1, -1, -1,],
]).reshape(-1)
VECTORS_WEIGHTS_NEW = np.array([
    [1/36, 1/9, 1/36,],
    [1/9,  4/9, 1/9,],
    [1/36, 1/9, 1/36,],
]).reshape(-1)
LAT_LEFT_COL_SL_NEW = np.s_[:, [0, 3, 6]]
LAT_CENT_COL_SL_NEW = np.s_[:, [1, 4, 7]]
LAT_RIGHT_COL_SL_NEW= np.s_[:, [2, 5, 8]]
# 'NW' 'N' 'NE' 'W' 'C' 'E' 'SW' 'S' 'SE'
#   0   1    2   3   4   5    6   7    8
VECTORS_DIRECTIONS_NEW = np.array([
    ['NW', 'N', 'NE',],
    ['W',  'C',  'E',],
    ['SW', 'S', 'SE',],
]).reshape(-1)
assert np.all(VECTORS_DIRECTIONS_NEW[LAT_LEFT_COL_SL_NEW[1]] == LEFT_COL_NAMES)
assert np.all(VECTORS_DIRECTIONS_NEW[LAT_CENT_COL_SL_NEW[1]] == CENT_COL_NAMES)
assert np.all(VECTORS_DIRECTIONS_NEW[LAT_RIGHT_COL_SL_NEW[1]] == RIGHT_COL_NAMES)
CYL_BOUNCE_BACK_DIRECTIONS_NEW = [8, 7, 6, 5, 4, 3, 2, 1, 0]

### cmp old and new

In [7]:
# check that new and old style colums slices are the same
assert np.all(
    VECTORS_DIRECTIONS_NEW[LAT_LEFT_COL_SL_NEW[1]] ==
    VECTORS_DIRECTIONS_OLD[LAT_LEFT_COL_SL_OLD[1]]
)
assert np.all(
    VECTORS_DIRECTIONS_NEW[LAT_CENT_COL_SL_NEW[1]] ==
    VECTORS_DIRECTIONS_OLD[LAT_CENT_COL_SL_OLD[1]]
)
assert np.all(
    VECTORS_DIRECTIONS_NEW[LAT_RIGHT_COL_SL_NEW[1]] ==
    VECTORS_DIRECTIONS_OLD[LAT_RIGHT_COL_SL_OLD[1]]
)

In [8]:
# Helper call to convert old lattices to the new one
OLD_TO_NEW_INDEXES = [list(VECTORS_DIRECTIONS_OLD).index(item) for item in VECTORS_DIRECTIONS_NEW]
NEW_TO_OLD_INDEXES = [list(VECTORS_DIRECTIONS_NEW).index(item) for item in VECTORS_DIRECTIONS_OLD]

In [9]:
# check that old and new coordinates are the same
assert (VECTORS_DIRECTIONS_NEW == VECTORS_DIRECTIONS_OLD[OLD_TO_NEW_INDEXES]).all()
assert (VECTORS_DIRECTIONS_OLD == VECTORS_DIRECTIONS_NEW[NEW_TO_OLD_INDEXES]).all()
assert (VECTORS_DIRECTIONS_OLD == VECTORS_DIRECTIONS_OLD[OLD_TO_NEW_INDEXES][NEW_TO_OLD_INDEXES]).all()

assert (VECTORS_VELOCITIES_X_NEW[NEW_TO_OLD_INDEXES] == VECTORS_VELOCITIES_X_OLD).all()
assert (VECTORS_VELOCITIES_Y_NEW[NEW_TO_OLD_INDEXES] == VECTORS_VELOCITIES_Y_OLD).all()
assert (VECTORS_WEIGHTS_NEW[NEW_TO_OLD_INDEXES] == VECTORS_WEIGHTS_OLD).all()

### Choose variables style

In [10]:
# style = "NEW"
# LATICE_VELOCITY_X = VELOCITIES_X = eval(f"VECTORS_VELOCITIES_X_{style}")
# LATICE_VELOCITY_Y = VELOCITIES_Y = eval(f"VECTORS_VELOCITIES_Y_{style}")
# WEIGHTS = WEIGHTS_MAT = eval(f"VECTORS_WEIGHTS_{style}")
# LAT_LEFT_COL_SL = eval(f"LAT_LEFT_COL_SL_{style}")
# LAT_CENT_COL_SL = eval(f"LAT_CENT_COL_SL_{style}")
# LAT_RIGHT_COL_SL = eval(f"LAT_RIGHT_COL_SL_{style}")
# VECTORS_DIRECTIONS = eval(f"VECTORS_DIRECTIONS_{style}")
# CYL_BOUNCE_BACK_DIRECTIONS = eval(f"CYL_BOUNCE_BACK_DIRECTIONS_{style}")

In [11]:
style = "NEW"
VECTORS_VELOCITIES_X = eval(f"VECTORS_VELOCITIES_X_{style}")
VECTORS_VELOCITIES_Y = eval(f"VECTORS_VELOCITIES_Y_{style}")
VECTORS_WEIGHTS = eval(f"VECTORS_WEIGHTS_{style}")
LAT_LEFT_COL_SL = eval(f"LAT_LEFT_COL_SL_{style}")
LAT_CENT_COL_SL = eval(f"LAT_CENT_COL_SL_{style}")
LAT_RIGHT_COL_SL = eval(f"LAT_RIGHT_COL_SL_{style}")
VECTORS_DIRECTIONS = eval(f"VECTORS_DIRECTIONS_{style}")
CYL_BOUNCE_BACK_DIRECTIONS = eval(f"CYL_BOUNCE_BACK_DIRECTIONS_{style}")

# Predefined funstions
## Tensorflow helper methods

## Initial contisions func

In [12]:
### Initial conditions
def init_random_cos():
    # F.shape: (100, 400, 9)
    F = np.ones((LENGTH_Y, LENGTH_X, N_VECTORS)) #* RHO_0 / N_VECTORS
    np.random.seed(42)
    F += 0.01 * np.random.randn(LENGTH_Y, LENGTH_X, N_VECTORS)
    X, _ = np.meshgrid(range(LENGTH_X), range(LENGTH_Y))
    # F[0, 0] - 0.99 .. 3.45
    F[:, :, 3] += 2 * (1 + 0.2 * np.cos(2 * np.pi * X / LENGTH_X * 4))  # 1.6..2.4
    rho = np.sum(F, 2)
    for i in VECTOR_INDEXES:
        F[:, :, i] *= RHO_0 / rho
    # F[0, 0] - 8 .. 29
    return F

def poiseuille_profile(y_phys):
    return 4 * U_MAX / (PIPE_LENGTH ** 2) * (y_phys * PIPE_LENGTH - y_phys * y_phys)

def init_poiseuille():
    rho = 1
    y, x = np.meshgrid(np.arange(LENGTH_Y), np.arange(LENGTH_X))
    F = np.empty((LENGTH_X, LENGTH_Y, N_VECTORS))
    y_phys = y - 0.5;
    ux = poiseuille_profile(y_phys)
    uy = np.zeros((LENGTH_X, LENGTH_Y))
    
    for idx in range(9):
        # 300, 100
        cu = 3 * (VECTORS_VELOCITIES_X[idx] * ux + VECTORS_VELOCITIES_Y[idx] * uy);
        # 300, 100
        res = rho * VECTORS_WEIGHTS[idx] * (1 + cu + 1/2 * cu ** 2 - 3/2*(ux**2 + uy **2));
        F[:, :, idx] = res
    F = np.rot90(F)
    return F

## Equality functions
### Old style

In [13]:
def calc_ba_eq(F):
    F_eq = np.zeros(F.shape)
    for i, cx, cy, w in zip(VECTOR_INDEXES, VECTORS_VELOCITIES_X, VECTORS_VELOCITIES_Y, VECTORS_WEIGHTS):
        # (100, 400)
        F_eq[:,:,i] = rho * w * ( 1 + 3*(cx*ux+cy*uy)  + 9*(cx*ux+cy*uy)**2/2 - 3*(ux**2+uy**2)/2 )
    print_v("F_eq_ba:\n", F_eq[0][0])
    return F_eq

### np equality

In [14]:
def calc_np_eq(data_flat):
    F = data_flat
    F_eq_l = np.zeros(F.shape)
    for x_idx in range(F.shape[0]):
        for y_idx in range(F.shape[1]):
            lattice = F[x_idx, y_idx, :]
            rho_l = np.sum(lattice)
            ux_l = np.sum(lattice * VECTORS_VELOCITIES_X) / rho_l
            uy_l = np.sum(lattice * VECTORS_VELOCITIES_Y) / rho_l
            u_sum = VECTORS_VELOCITIES_X * ux_l + VECTORS_VELOCITIES_Y * uy_l
            F_eq_lattice = rho_l * VECTORS_WEIGHTS * (
                1 + 3 * (u_sum) + 9 * (u_sum) ** 2 / 2 - 3 * (ux_l ** 2 + uy_l ** 2) / 2
            )
            F_eq_l[x_idx, y_idx, :] = F_eq_lattice
    print_v("F_eq_np:\n", F_eq_l[0][0])
    return F_eq_l

### tf equality

In [15]:
### Tensorflow helper methods
def build_graph():
    dtype = tf.float32
    velocities_x_tf = tf.constant(VECTORS_VELOCITIES_X, dtype=dtype)
    velocities_y_tf = tf.constant(VECTORS_VELOCITIES_Y, dtype=dtype)
    weights_tf = tf.constant(VECTORS_WEIGHTS, dtype=dtype)
    
    def ones_init(shape, dtype=None, partition_info=None):
        kernel = np.zeros(shape)
        kernel[0, 0, :, 0] = 1.0
        return tf.cast(kernel, dtype)

    sum_conv = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer=ones_init)

    def vel_x_init_many_to_one(shape, dtype=None, partition_info=None):
        kernel = np.zeros(shape)
        kernel[0, 0, :, 0] = VECTORS_VELOCITIES_X
        return tf.cast(kernel, dtype)

    vel_x_conv = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer=vel_x_init_many_to_one)


    def vel_y_init_many_to_one(shape, dtype=None, partition_info=None):
        kernel = np.zeros(shape)
        kernel[0, 0, :, 0] = VECTORS_VELOCITIES_Y
        return tf.cast(kernel, dtype)

    vel_y_conv = tf.keras.layers.Conv2D(1, (1, 1), kernel_initializer=vel_y_init_many_to_one)
    return velocities_x_tf, velocities_y_tf, weights_tf, sum_conv, vel_x_conv, vel_y_conv

In [19]:
def calc_tf_eq_core(data_flat, tf_params):
    velocities_x_tf, velocities_y_tf, weights_tf, sum_conv, vel_x_conv, vel_y_conv = tf_params
#     batch = data_flat.reshape(1, *data_flat.shape)
    batch = data_flat
#     rho = sum_conv(batch)
    rho = tf.cast(tf.expand_dims(tf.math.reduce_sum(batch, axis=3), axis=-1), tf.float32)
    ux_lattices = vel_x_conv(batch) / rho
    uy_lattices = vel_y_conv(batch) / rho
    ux_elements = tf.math.multiply(ux_lattices, velocities_x_tf)
    uy_elements = tf.math.multiply(uy_lattices, velocities_y_tf)
    before_weights = (
        1 + 3 * (ux_elements + uy_elements) +
        9 * (ux_elements + uy_elements) ** 2 / 2 - 
        3 * (ux_lattices ** 2 + uy_lattices ** 2) / 2
    )
    after_weights = tf.math.multiply(before_weights, weights_tf)
    F_eq = tf.math.multiply(rho, after_weights)
#     F_eq = F_eq.numpy().squeeze()
#     print_v("F_eq_tf:\n", F_eq[0][0])
    return F_eq

calc_tf_eq_func = tf.function(calc_tf_eq_core)
TF_PARAMS = build_graph()

def calc_tf_eq(data_flat):
    batch = data_flat.reshape(1, *data_flat.shape)
    return calc_tf_eq_func(batch, TF_PARAMS).numpy().squeeze()

### Optimized tf

In [17]:
from tensorflow.keras import Input, Model

batch = Input(shape=(LENGTH_Y, LENGTH_Y, N_VECTORS))
velocities_x_tf, velocities_y_tf, weights_tf, sum_conv, vel_x_conv, vel_y_conv = build_graph()
# rho = sum_conv(batch)
rho = tf.expand_dims(tf.math.reduce_sum(batch, axis=3), axis=-1)
ux_lattices = vel_x_conv(batch) / rho
uy_lattices = vel_y_conv(batch) / rho
ux_elements = tf.math.multiply(ux_lattices, velocities_x_tf)
uy_elements = tf.math.multiply(uy_lattices, velocities_y_tf)
before_weights = (
    1 + 3 * (ux_elements + uy_elements) +
    9 * (ux_elements + uy_elements) ** 2 / 2 - 
    3 * (ux_lattices ** 2 + uy_lattices ** 2) / 2
)
after_weights = tf.math.multiply(before_weights, weights_tf)
F_eq = tf.math.multiply(rho, after_weights)
model = Model(inputs=batch, outputs=F_eq)
# graph_model = tf.function(model)
tf.saved_model.save(model, '/tmp/model')
loaded = tf.saved_model.load('/tmp/model')
infer = loaded.signatures["serving_default"]

def predict(data):
    data_batch = data.reshape(1, *data.shape)
    tf_res = infer(tf.constant(data_batch, dtype=tf.float32))
    np_res = tf_res[model.output_names[0]].numpy().squeeze()
    return np_res

2021-11-08 04:44:52.176412: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/model/assets


# Simulation loop

In [20]:
### Simulation(main loop)
cmap = plt.cm.bwr.copy()
cmap.set_bad('black')

plot = False
# PRINT = not plot
PRINT = False
macroscopic = False
if plot:
    fig, axes = plt.subplots(2, figsize=(10, 8))
    for ax in axes:
        ax.invert_yaxis()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.set_aspect('equal')
    fig.tight_layout()

# F = init_random_cos()
F = init_poiseuille()

def print_v(*data):
    if not plot and PRINT:
        print(*data)

MODE = 'tf'
# old - 00:00:47
# np -  01:30:45
# tf -  00:01:40
# tf -  00:00:56  # function optimized
# tfopt 00:00:56  # Optimized via saving
for it in tqdm(range(N_STEPS), disable=plot):
# for it in range(10):

    # Set reflective boundaries
    bndryF = F[CYLINDER_MASK,:]   # shape: (3405, 9)
    # Action: to all cylinder coordinates assign such f_i values
    # FIXME: where do we get that array???
    # NOTE: should be fixed after boundary conditions implementation
#     CYL_BOUNCE_BACK_DIRECTIONS = [4, 5, 6, 3, 0, 7, 2, 1, 8]
#     CYL_BOUNCE_BACK_DIRECTIONS = [0, 5, 6, 7, 8, 1, 2, 3, 4]
    bndryF = bndryF[:, CYL_BOUNCE_BACK_DIRECTIONS]

    ### 1. Compute moments (for each latice)
    rho = np.sum(F, 2)  # shape: (100, 400)
    ux  = np.sum(F * VECTORS_VELOCITIES_X, 2) / rho   # shape: (100, 400)
    uy  = np.sum(F * VECTORS_VELOCITIES_Y, 2) / rho   # shape: (100, 400)
    
    if macroscopic:
        ### 1.1 Compute macroscopic (dirichlet) boundary conditions
        ##  Inlet: Poiseuille profile
        y_phys = np.arange(LENGTH_Y) - 0.5
        ux[INLET_SL] = poiseuille_profile(y_phys)
        uy[INLET_SL] = 0
        rho[INLET_SL] = 1 / (1 - ux[INLET_SL] * (
            F[INLET_SL][LAT_CENT_COL_SL].sum(axis=1) +
            2 * F[INLET_SL][LAT_LEFT_COL_SL].sum(axis=1))
        )
        ##  Outlet: Constant pressure
        print_v('-')
        print_v("rho outlet before", rho[OUTLET_SL].mean())
        rho[OUTLET_SL] = 1
        print_v("rho outlet after", rho[OUTLET_SL].mean())
        print_v("rho total", rho.mean())
        print_v("F[OUTLET_SL][LAT_CENT_COL_SL].sum(axis=1)", np.mean(F[OUTLET_SL][LAT_CENT_COL_SL].sum(axis=1)))
        print_v("2 * F[OUTLET_SL][LAT_RIGHT_COL_SL].sum(axis=1)", np.mean(2 * F[OUTLET_SL][LAT_RIGHT_COL_SL].sum(axis=1)))

        print_v("ux before", ux[OUTLET_SL].mean())
        ux[OUTLET_SL] = -1 + 1 / rho[OUTLET_SL] * (
            F[OUTLET_SL][LAT_CENT_COL_SL].sum(axis=1) +
            2 * F[OUTLET_SL][LAT_RIGHT_COL_SL].sum(axis=1)
        )
        print_v("ux after", ux[OUTLET_SL].mean())
        uy[OUTLET_SL] = 0

    ### 2. Compute equilibrium
    if MODE == 'old':
        F_eq = calc_ba_eq(F)
    elif MODE == 'np':
        F_eq = calc_np_eq(F)
    elif MODE == 'tf':
        F_eq = calc_tf_eq(F)
    elif MODE == 'tf_opt':
        F_eq = predict(F)

    ### 3. Collide locally
    F = F - (F - F_eq) / TAU

    # Apply boundary 
    # IDEA: copy that strange random array which was applied to a cylinder
    F[CYLINDER_MASK, :] = bndryF
    
    ### 4. Propagate to the neighbours
    for i, cx, cy in zip(VECTOR_INDEXES, VECTORS_VELOCITIES_X, VECTORS_VELOCITIES_Y):
        F[:, :, i] = np.roll(F[:, :, i], (cx, cy), axis=(1, 0))
        

    if (it % 10 == 0 or it == N_STEPS - 1) and plot:
        ux[CYLINDER_MASK] = 0
        uy[CYLINDER_MASK] = 0
        # Note: Calculate vorticity as a difference 
        # Note: np.roll(ux, -1, axis=0) - shift all X velocity by one ROW down
        # vorticity.shape: (100, 400)
        vorticity = (
            (np.roll(ux, -1, axis=0) - np.roll(ux, 1, axis=0)) - 
            (np.roll(uy, -1, axis=1) - np.roll(uy, 1, axis=1))
        )
        vorticity[CYLINDER_MASK] = np.nan
        
        # display velocity
        rho = np.sum(F, 2)
        ux  = np.sum(F * VECTORS_VELOCITIES_X, 2) / rho
        uy  = np.sum(F * VECTORS_VELOCITIES_Y, 2) / rho
        u = np.sqrt(ux ** 2 + uy ** 2)
        axes[0].cla()
        axes[0].set_title("Velocity")
        axes[0].imshow(u)
        
        # Works but blink all the time
        axes[1].cla()
        axes[1].set_title("Vorticity")
        axes[1].imshow(vorticity, cmap=cmap)
        axes[1].get_images()[0].set_clim(-.1, .1)
        display(fig)
        clear_output(wait=True)
        plt.pause(0.001)

 18%|███████████▏                                                 | 733/4000 [00:10<00:46, 70.20it/s]


KeyboardInterrupt: 